In [47]:
import sympy as sp
from sympy import pprint
import numpy as np
import pandas as pd

In [48]:
#INITIALIZE VARIABLES

l1 = sp.Symbol('l_1')
l2 = sp.Symbol('l_2')
l3 = sp.Symbol('l_3')
l4 = sp.Symbol('l_4')
theta1 = sp.Symbol('theta_1')
theta2 = sp.Symbol('theta_2')
theta3 = sp.Symbol('theta_3')



In [49]:
#INITIALIZE SYMBOLS
thetai = sp.Symbol('theta_i')
t = sp.Symbol('t')
ai = sp.Symbol('a_i')
di = sp.Symbol('d_i')
alphai = sp.Symbol('alpha_i')
pi = sp.Symbol('pi')

In [50]:
#CREATE DH TABLE ROWS


dh_theta = [theta1,(pi/2)+theta2, theta3-(pi/3), 0]
dh_a = [0,0,0,0]
dh_d = [l1,l2,0,l3+l4]
dh_alpha = [-pi/6,pi/2,-pi/2,0]


In [51]:
#CREATE DH TABLE

dh_table = {
    "": ["0 to 1", "1 to 2", "2 to 3", "3to 4"],
    di: [dh_d[0],dh_d[1],dh_d[2],dh_d[3]],
    thetai: [dh_theta[0],dh_theta[1],dh_theta[2],dh_theta[3]],
    alphai: [dh_alpha[0],dh_alpha[1],dh_alpha[2],dh_alpha[3]],
    ai: [dh_a[0],dh_a[1],dh_a[2],dh_a[3]]

}

df = pd.DataFrame(dh_table)
df

,,d_i,theta_i,alpha_i,a_i
0,0 to 1,l_1,theta_1,-pi/6,0
1,1 to 2,l_2,pi/2 + theta_2,pi/2,0
2,2 to 3,0,-pi/3 + theta_3,-pi/2,0
3,3to 4,l_3 + l_4,0,0,0


In [52]:
#CREATE SYMBOLIC TRANSFORMATION MATRICES
#Array of all transformation matrices
A_array = [sp.zeros(4,4)]*4
for i in range(0,4):
    A_array[i] = sp.Matrix([
        [sp.cos(dh_theta[i]),-sp.sin(dh_theta[i])*sp.cos(dh_alpha[i]),sp.sin(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.cos(dh_theta[i])],
        [sp.sin(dh_theta[i]),sp.cos(dh_theta[i])*sp.cos(dh_alpha[i]),-sp.cos(dh_theta[i])*sp.sin(dh_alpha[i]),dh_a[i]*sp.sin(dh_theta[i])],
        [0,sp.sin(dh_alpha[i]),sp.cos(dh_alpha[i]),dh_d[i]],
        [0,0,0,1]
    ])
    print("Transformation matrix: Frame ", i ," to ", i+1)
    pprint(A_array[i])
    print("----------------------------------------------------------")
    






Transformation matrix: Frame  0  to  1
⎡                     ⎛π⎞      ⎛π⎞            ⎤
⎢cos(θ₁)  -sin(θ₁)⋅cos⎜─⎟  -sin⎜─⎟⋅sin(θ₁)  0 ⎥
⎢                     ⎝6⎠      ⎝6⎠            ⎥
⎢                                             ⎥
⎢            ⎛π⎞              ⎛π⎞             ⎥
⎢sin(θ₁)  cos⎜─⎟⋅cos(θ₁)   sin⎜─⎟⋅cos(θ₁)   0 ⎥
⎢            ⎝6⎠              ⎝6⎠             ⎥
⎢                                             ⎥
⎢                 ⎛π⎞             ⎛π⎞         ⎥
⎢   0         -sin⎜─⎟          cos⎜─⎟       l₁⎥
⎢                 ⎝6⎠             ⎝6⎠         ⎥
⎢                                             ⎥
⎣   0            0                0         1 ⎦
----------------------------------------------------------
Transformation matrix: Frame  1  to  2
⎡   ⎛π     ⎞      ⎛π     ⎞    ⎛π⎞     ⎛π⎞    ⎛π     ⎞     ⎤
⎢cos⎜─ + θ₂⎟  -sin⎜─ + θ₂⎟⋅cos⎜─⎟  sin⎜─⎟⋅sin⎜─ + θ₂⎟   0 ⎥
⎢   ⎝2     ⎠      ⎝2     ⎠    ⎝2⎠     ⎝2⎠    ⎝2     ⎠     ⎥
⎢                                                         ⎥

In [53]:
#MULTIPLY TRANSFORMATION MATRICES
#sub in numerical value for pi
for i in range(0,4):
    A_array[i] = A_array[i].subs([(pi,np.pi)])
    
#Final transformation matrix A1*A2*A3*A4
A_final = A_array[0]
for i in range(1,4):
    A_final = A_final*A_array[i]


#Array of transformation matrix products [A1, A1*A2, A1*A2*A3,] etc
A_products = [A_array[0]]*4
for i in range(1,4):
    A_products[i] = A_products[i-1]*A_array[i]

In [54]:
#MATRIX SUBSTITUTION FUNCTIONS

#Substitutes theta values into given matrix and returns matrix 
def subs_function(matrix,sub_values):
    result = matrix.subs([
        (theta1,sub_values[0]),
        (theta2,sub_values[1]),
        (theta3,sub_values[2]),
        (l4,sub_values[3])   
    ])
    return result

#Returns position vector of given transformation matrix and theta values
def subs_position_vector(matrix,sub_values):
    msubs = matrix.subs([
        (theta1,sub_values[0]),
        (theta2,sub_values[1]),
        (theta3,sub_values[2]),
        (l4,sub_values[3])   
    ])
    result = sp.Matrix([
        [msubs[0,3]],
        [msubs[1,3]],
        [msubs[2,3]],
    ])
    return result






In [55]:


# #VALIDATE TRANSFORMATION MATRIX

# #5 different sets of theta values for validation
for i in range(0,4):
    A_products[i] = A_products[i].subs([(l1,1.5),(l2,1),(l3,0.5)])
    
A_final = A_final.subs([(l1,1.5),(l2,1),(l3,0.5)])

q = 3.1415/2
val_matrix = sp.Matrix([
    [0, q, 0, 0, 0],
    [0, 0, q, 0, 0],
    [0, 0, 0, q, 0],
    [0, 0, 0, 0, 2]

])

#Array of resulting matrices
A_vals = [sp.zeros(4,4)]*5
for i in range(0,5):
    A_vals[i] = subs_function(A_final,val_matrix[:,i])


pos_vals = ["start position","first joint rotated 90 degrees","second joint rotated 90 degrees","third joint rotated 90 degrees","last joint extended 2"]
print("Transformation matrix validation")
for i in range(0,5):
    print("---------------")
    print(pos_vals[i])
    print(
        "x = ",((A_vals[i])[0,3]).round(2),#get x value from matrix i in A_vals array
        "y = ",((A_vals[i])[1,3]).round(2),#get y value from matrix i in A_vals array
        "z = ",((A_vals[i])[2,3]).round(2))#get z value from matrix i in A_vals array




Transformation matrix validation
---------------
start position
x =  0.0 y =  1.00000000000000 z =  2.37
---------------
first joint rotated 90 degrees
x =  -1.00000000000000 y =  0.0 z =  2.37
---------------
second joint rotated 90 degrees
x =  -0.43 y =  0.63 z =  2.58
---------------
third joint rotated 90 degrees
x =  0.0 y =  0.50 z =  2.87
---------------
last joint extended 2
x =  0.0 y =  3.00000000000000 z =  2.37


In [56]:
#PROBLEM 1D
point = sp.Matrix([-1,2,3,1])
given_positions = [np.pi/2,-np.pi/6,np.pi/3,2]
T_4_P = sp.Matrix([
    [1,0,0,-1],
    [0,1,0,2],
    [0,0,1,3],
    [0,0,0,1]
])
# subs_function(A_final,new_positions)*point
# subs_function(A_final,new_positions)*new_matrix
subs_position_vector(A_final*T_4_P,given_positions)

Matrix([
[-3.36602540378444],
[-2.23205080756888],
[ 7.06217782649107]])

In [57]:
#JACOBIAN


In [66]:
#CREATE JACOBIAN


# thetas_vector = sp.Matrix([thetas[0],thetas[1],thetas[2],thetas[3],thetas[4],thetas[5]])

# P = sp.Matrix([A_final[0,3],A_final[1,3],A_final[2,3]])

# Jv = P.jacobian(thetas_vector) #create 3x6 Jv matrix

Z = sp.zeros(3,5) #initialize 3x6 Jw matrix of zeros
Z[:,0] = sp.Matrix([0,0,1])

#replace values in Z with correct values from matrices in A_products array
for c in range(1,5):
    for r in range(0,3):
        Z[r,c] = (A_products[c-1])[r,2]


O = sp.zeros(3,5)
O[:,0] = sp.Matrix([0,0,0])
for c in range(1,4):
    for r in range(0,3):
        O[r,c] = (A_products[c-1])[r,3]

print("Z components:-------------------------------------")
pprint(Z)
print("O components:-------------------------------------")
pprint(O)




Z components:-------------------------------------
⎡0    -0.5⋅sin(θ₁)     0.866025403784439⋅sin(θ₁)⋅cos(θ₂ + 1.5707963267949) - 3 ↪
⎢                                                                              ↪
⎢0     0.5⋅cos(θ₁)     1.0⋅sin(θ₁)⋅sin(θ₂ + 1.5707963267949) - 0.8660254037844 ↪
⎢                                                                              ↪
⎣1  0.866025403784439                                     0.5⋅cos(θ₂ + 1.57079 ↪

↪ .06161699786838e-17⋅sin(θ₁) + 1.0⋅sin(θ₂ + 1.5707963267949)⋅cos(θ₁)  -1.0⋅(- ↪
↪                                                                              ↪
↪ 39⋅cos(θ₁)⋅cos(θ₂ + 1.5707963267949) + 3.06161699786838e-17⋅cos(θ₁)  -1.0⋅(s ↪
↪                                                                              ↪
↪ 63267949) + 5.30287619362453e-17                                             ↪

↪ 0.866025403784439⋅sin(θ₁)⋅sin(θ₂ + 1.5707963267949) + cos(θ₁)⋅cos(θ₂ + 1.570 ↪
↪                                                       

In [71]:
#PROBLEM 4
theta = sp.Symbol('theta')
h = sp.Symbol('h')
l = sp.Symbol('l')
T_0_1_inv = (1/2)*sp.Matrix([
    [2*sp.sin(theta),-2*sp.cos(theta),0,-2*h],
    [sp.cos(theta)*sp.sqrt(2),sp.sin(theta)*sp.sqrt(2),sp.sqrt(2),-l*sp.sqrt(2)],
    [-sp.cos(theta)*sp.sqrt(2),-sp.sin(theta)*sp.sqrt(2),sp.sqrt(2),-l*sp.sqrt(2)],
    [0,0,0,2]
])
T_0_1 = sp.simplify(T_0_1_inv.inv())
pprint(T_0_1)
alpha = sp.asin(sp.sqrt(2)/2)


⎡1.0⋅sin(θ)   0.5⋅√2⋅cos(θ)  -0.5⋅√2⋅cos(θ)  1.0⋅h⋅sin(θ) ⎤
⎢                                                         ⎥
⎢-1.0⋅cos(θ)  0.5⋅√2⋅sin(θ)  -0.5⋅√2⋅sin(θ)  -1.0⋅h⋅cos(θ)⎥
⎢                                                         ⎥
⎢     0          0.5⋅√2          0.5⋅√2          1.0⋅l    ⎥
⎢                                                         ⎥
⎣     0             0              0              1.0     ⎦
